In [ ]:
import tabula
import pandas as pd

In [12]:
path = '../data/raw/NYSDOL_debarment_02_19_2025.pdf'

pages = tabula.read_pdf(path, stream = True, pages='all')

In [13]:
import numpy as np

cols = ['AGENCY','FISCAL_OFFICER','FEIN','EMPLOYER_NAME','EMPLOYER_DBA','ADDRESS','DEBAR_START','DEBAR_END']

df = pd.DataFrame(columns=cols)
for pg in pages:
    if pg.shape[1] == 7:
        pg.insert(4, '', np.nan)
    coldf = pd.DataFrame(pg.columns).T
    coldf.columns = cols
    pg.columns = cols
    df = pd.concat([df,coldf,pg],ignore_index=True)


In [14]:
df.reset_index(drop=True, inplace=True)
records = df.loc[df['AGENCY'].notna()].reset_index().reset_index().rename(columns={'level_0':'ID'}).set_index('index')

In [15]:
df = df.join(records[['ID']],how='left')
df['ID'] = df['ID'].ffill()

In [16]:
df.shape

(446, 9)

In [17]:
df.fillna('',inplace=True)
df = df.groupby('ID').agg(lambda x: " ".join([i for i in list(x) if i])).reset_index()
df = df.replace(r'Unnamed:\s\d+', '', regex=True)

In [18]:
df = df.drop(df[df['AGENCY'] == 'AGENCY'].index).reset_index(drop=True)

df

,ID,AGENCY,FISCAL_OFFICER,FEIN,EMPLOYER_NAME,EMPLOYER_DBA,ADDRESS,DEBAR_START,DEBAR_END
0,1.0,DOL,DOL,*****5754,"0369 CONTRACTORS, LLC",,515 WEST AVE UNIT PH 13NORWALK CT 06850,05/12/2021,05/12/2026
1,2.0,DOL,DOL,*****5784,"A.J.M. TRUCKING, INC.",,PO BOX 2064 MONROE NY 10950,02/12/2024,02/12/2029
2,3.0,DOL,DOL,,AKHLAQ OULAKH,,4307 28TH AVE ASTORIA NY 11103,10/11/2024,10/11/2029
3,4.0,DOL,NYC,,"ALL COUNTY SEWER & DRAIN, INC.",,7 GREENFIELD DR WARWICK NY 10990,03/25/2022,03/25/2027
4,5.0,DOL,DOL,*****8387,"AMERICAN PAVING & MASONRY, CORP.",,8 FOREST AVE GLEN COVE NY 11542,05/24/2024,05/24/2029
...,...,...,...,...,...,...,...,...,...
202,204.0,DOL,DOL,,PAULINE CHAHALES,,935 S LAKE BLVD MAHOPAC NY 10541,05/17/2021,05/17/2026
203,205.0,DOL,DOL,*****9060,"PEC GROUP OF N.Y., INC.",,935 S LAKE BLVD SUITE 7MAHOPAC NY 10541,05/17/2021,05/17/2026
204,206.0,DOL,DOL,*****9060,"PEC GROUP OF N.Y., INC.",,935 S LAKE BLVD SUITE 7MAHOPAC NY 10541,03/02/2021,03/02/2026
205,207.0,DOL,DOL,,RUSSELL NEEDHAM,,532 NEPTUNE AVENUE BROOKLYN NY 11224,01/12/2022,01/12/2027


In [20]:
df.to_csv('../data/processed/NYSDOL_debarment_02_19_2025.csv',index=False)